# Python Intro - Final Exercise


In [2]:
import pyodbc
import pandas as pd
import numpy as np

In [5]:
### In windows, Using a ODBC DNS (predefined connection name) ------didn't work
#conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')

#conn = pyodbc.connect("DSN=COLLEGE;"
#                      "Server=DESKTOP-4P3K6TB\SQLEXPRESS01;" ------didn't work
#                      "Database=COLLEGE;"
#                      "Trusted_Connection=yes;")

#conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
#                      "Server=DESKTOP-4P3K6TB\SQLEXPRESS01;"
#                      "Database=COLLEGE;"
#                      "Trusted_Connection=yes;")

conn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=192.168.1.1;"
                      "Database=COLLEGE;"
                      "uid=dsuser06;"
                      "pwd=DSuser06!;"
                     )


In [6]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 

In [7]:
### Get the classrooms table
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn) 

In [8]:
### Get the courses table
courses = pd.read_sql("SELECT * FROM Courses;",conn) 

In [9]:
### Get the departments table
departments = pd.read_sql("SELECT * FROM Departments;",conn) 

In [10]:
### Get the teachers table
teachers = pd.read_sql("SELECT * FROM Teachers;",conn) 

In [11]:
co_cl_dep = pd.merge(pd.merge(courses, classrooms, on='CourseId', how='inner'), departments, left_on='DepartmentID', right_on='DepartmentId')

# Questions
 
 
## Q1. Count the number of students on each department

In [12]:
co_cl_dep.head()

CourseId         CourseName  DepartmentID  TeacherId  StudentId  grades  \
0         1  English Begginers             1         15        202    92.0   
1         1  English Begginers             1         15         12    81.2   
2         1  English Begginers             1         15        232    82.4   
3         1  English Begginers             1         15        123    62.4   
4         1  English Begginers             1         15        274    72.6   

   DepartmentId DepartmentName  
0             1        English  
1             1        English  
2             1        English  
3             1        English  
4             1        English

In [13]:
dep_count = co_cl_dep.loc[:, ['DepartmentName','StudentId']].drop_duplicates().groupby('DepartmentName').agg({'StudentId':['count']})

In [14]:
dep_count

StudentId
                   count
DepartmentName          
Arts                 137
English               84
Science              200
Sport                 81

## Q2. How many students have each course of the English department and the total number of students in the department?

In [15]:
co_count_eng = co_cl_dep.loc[(co_cl_dep['DepartmentID'] == 1),
              ['CourseName','StudentId']].drop_duplicates().groupby('CourseName').agg({'StudentId':['count']})


In [16]:
co_count_eng

StudentId
                         count
CourseName                    
Advanced English            24
English Begginers           32
Proffesional English        38

In [17]:
co_count_eng.loc['Total'] = co_cl_dep.loc[(co_cl_dep['DepartmentID'] == 1),
              'StudentId'].drop_duplicates().agg({'StudentId':['count']})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  


In [18]:
co_count_eng

StudentId
                         count
CourseName                    
Advanced English            24
English Begginers           32
Proffesional English        38
Total                       84

## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [19]:
co_count_sci = co_cl_dep.loc[(co_cl_dep['DepartmentID'] == 2),
              ['CourseName','StudentId']].drop_duplicates().groupby('CourseName').agg({'StudentId':['count']})

In [20]:
co_count_sci.loc[(co_count_sci['StudentId']['count'] < 22), 'ClassroomSize'] = 'Small'
co_count_sci.loc[(co_count_sci['StudentId']['count'] >= 22), 'ClassroomSize'] = 'Big'

In [21]:
co_count_sci

StudentId ClassroomSize
                        count              
CourseName                                 
Algebra                    12         Small
Biology                    36           Big
Chemistry                  23           Big
Computer Science           38           Big
Data Science               30           Big
Genetics                   37           Big
Geometry                   13         Small
Java                       16         Small
Matematics Advanced        10         Small
Mathematics 101            18         Small
Neurobiology               24           Big
Physics                    12         Small
Python                     23           Big
R                          33           Big
SQL                        24           Big
Statistics                 24           Big

In [22]:
co_count_sci.groupby('ClassroomSize').agg({'ClassroomSize':['count']})

ClassroomSize
                      count
ClassroomSize              
Big                      10
Small                     6

## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [23]:
students.drop_duplicates().groupby('Gender').agg({'StudentId':['count']})

StudentId
           count
Gender          
F            165
M            115

## Q5. For which courses the percentage of male/female students is over 70%?

In [24]:
co_cl_dep_st = pd.merge(co_cl_dep, students, on='StudentId', how='inner')

In [25]:
co_cl_dep_st

CourseId         CourseName  DepartmentID  TeacherId  StudentId  grades  \
0           1  English Begginers             1         15        202    92.0   
1           4    Mathematics 101             2          2        202    73.4   
2           8          Chemistry             2         14        202    83.6   
3          18                  R             2         17        202    66.1   
4          20    History of Arts             3          8        202    64.0   
..        ...                ...           ...        ...        ...     ...   
749        29              Tenis             4         24        105    81.8   
750        30           Swimming             4         12        158    76.3   
751        31         Gymnastics             4         23        241    82.8   
752        31         Gymnastics             4         23        262    78.3   
753        31         Gymnastics             4         23        108    79.7   

     DepartmentId DepartmentName FirstName    LastName Gender  
0               1        English    Autumn       Major      F  
1               2        Science    Autumn       Major      F  
2               2        Science    Autumn       Major      F  
3               2        Science    Autumn       Major      F  
4               3           Arts    Autumn       Major      F  
..            ...            ...       ...         ...    ...  
749             4          Sport    Hannah      Wilkes      F  
750             4          Sport  Mitchell   Fellhauer      M  
751             4          Sport   Kaitlyn      Tuttle      F  
752             4          Sport      Luke  Van Cleave      M  
753             4          Sport     Kevin     Clifton      M  

[754 rows x 11 columns]

In [26]:
gen_co = co_cl_dep_st.loc[:,['CourseName','StudentId','Gender']].drop_duplicates().\
groupby(['CourseName','Gender']).agg({'StudentId':['count']})

In [42]:
gen_co

StudentId
                                count
CourseName           Gender          
Advanced English     F             13
                     M             11
Algebra              F              8
                     M              4
Biology              F             19
                     M             17
Chemistry            F             15
                     M              8
Computer Science     F             18
                     M             20
Data Science         F             12
                     M             18
English Begginers    F             20
                     M             12
Genetics             F             26
                     M             11
Geometry             F              8
                     M              5
Gymnastics           F             13
                     M              9
History of Arts      F             17
                     M             15
Java                 F             10
                     M              6
Literature           F             21
                     M             18
Matematics Advanced  F              3
                     M              7
Mathematics 101      F             11
                     M              7
Musicology           F             21
                     M              9
Neurobiology         F             11
                     M             13
Painting             F             19
                     M             15
Physics              F              6
                     M              6
Proffesional English F             19
                     M             19
Python               F             13
                     M             10
R                    F             20
                     M             13
SQL                  F             16
                     M              8
Sculpture            F             17
                     M              6
Statistics           F             16
                     M              8
Swimming             F              9
                     M              9
Tenis                F             13
                     M              5
Theater              F             13
                     M             13
Volleyball           F             24
                     M             12

In [48]:
gen_co.reset_index(inplace=True)
gen_co

index            CourseName Gender StudentId
                                          count
0      0      Advanced English      F        13
1      1      Advanced English      M        11
2      2               Algebra      F         8
3      3               Algebra      M         4
4      4               Biology      F        19
5      5               Biology      M        17
6      6             Chemistry      F        15
7      7             Chemistry      M         8
8      8      Computer Science      F        18
9      9      Computer Science      M        20
10    10          Data Science      F        12
11    11          Data Science      M        18
12    12     English Begginers      F        20
13    13     English Begginers      M        12
14    14              Genetics      F        26
15    15              Genetics      M        11
16    16              Geometry      F         8
17    17              Geometry      M         5
18    18            Gymnastics      F        13
19    19            Gymnastics      M         9
20    20       History of Arts      F        17
21    21       History of Arts      M        15
22    22                  Java      F        10
23    23                  Java      M         6
24    24            Literature      F        21
25    25            Literature      M        18
26    26   Matematics Advanced      F         3
27    27   Matematics Advanced      M         7
28    28       Mathematics 101      F        11
29    29       Mathematics 101      M         7
30    30            Musicology      F        21
31    31            Musicology      M         9
32    32          Neurobiology      F        11
33    33          Neurobiology      M        13
34    34              Painting      F        19
35    35              Painting      M        15
36    36               Physics      F         6
37    37               Physics      M         6
38    38  Proffesional English      F        19
39    39  Proffesional English      M        19
40    40                Python      F        13
41    41                Python      M        10
42    42                     R      F        20
43    43                     R      M        13
44    44                   SQL      F        16
45    45                   SQL      M         8
46    46             Sculpture      F        17
47    47             Sculpture      M         6
48    48            Statistics      F        16
49    49            Statistics      M         8
50    50              Swimming      F         9
51    51              Swimming      M         9
52    52                 Tenis      F        13
53    53                 Tenis      M         5
54    54               Theater      F        13
55    55               Theater      M        13
56    56            Volleyball      F        24
57    57            Volleyball      M        12

In [51]:
gen_co_t = co_cl_dep_st.loc[:,['CourseName','StudentId','Gender']].drop_duplicates().\
groupby(['CourseName']).agg({'StudentId':['count']})

In [52]:
gen_co_t.reset_index(inplace=True)

In [53]:
gen_co_t

CourseName StudentId
                             count
0       Advanced English        24
1                Algebra        12
2                Biology        36
3              Chemistry        23
4       Computer Science        38
5           Data Science        30
6      English Begginers        32
7               Genetics        37
8               Geometry        13
9             Gymnastics        22
10       History of Arts        32
11                  Java        16
12            Literature        39
13   Matematics Advanced        10
14       Mathematics 101        18
15            Musicology        30
16          Neurobiology        24
17              Painting        34
18               Physics        12
19  Proffesional English        38
20                Python        23
21                     R        33
22                   SQL        24
23             Sculpture        23
24            Statistics        24
25              Swimming        18
26                 Tenis        18
27               Theater        26
28            Volleyball        36

In [54]:
gen_co_m = pd.merge(gen_co_t, gen_co, on='CourseName', how='left')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [55]:
gen_co_m

CourseName StudentId_x index Gender StudentId_y
                               count                    count
0       Advanced English          24     0      F          13
1       Advanced English          24     1      M          11
2                Algebra          12     2      F           8
3                Algebra          12     3      M           4
4                Biology          36     4      F          19
5                Biology          36     5      M          17
6              Chemistry          23     6      F          15
7              Chemistry          23     7      M           8
8       Computer Science          38     8      F          18
9       Computer Science          38     9      M          20
10          Data Science          30    10      F          12
11          Data Science          30    11      M          18
12     English Begginers          32    12      F          20
13     English Begginers          32    13      M          12
14              Genetics          37    14      F          26
15              Genetics          37    15      M          11
16              Geometry          13    16      F           8
17              Geometry          13    17      M           5
18            Gymnastics          22    18      F          13
19            Gymnastics          22    19      M           9
20       History of Arts          32    20      F          17
21       History of Arts          32    21      M          15
22                  Java          16    22      F          10
23                  Java          16    23      M           6
24            Literature          39    24      F          21
25            Literature          39    25      M          18
26   Matematics Advanced          10    26      F           3
27   Matematics Advanced          10    27      M           7
28       Mathematics 101          18    28      F          11
29       Mathematics 101          18    29      M           7
30            Musicology          30    30      F          21
31            Musicology          30    31      M           9
32          Neurobiology          24    32      F          11
33          Neurobiology          24    33      M          13
34              Painting          34    34      F          19
35              Painting          34    35      M          15
36               Physics          12    36      F           6
37               Physics          12    37      M           6
38  Proffesional English          38    38      F          19
39  Proffesional English          38    39      M          19
40                Python          23    40      F          13
41                Python          23    41      M          10
42                     R          33    42      F          20
43                     R          33    43      M          13
44                   SQL          24    44      F          16
45                   SQL          24    45      M           8
46             Sculpture          23    46      F          17
47             Sculpture          23    47      M           6
48            Statistics          24    48      F          16
49            Statistics          24    49      M           8
50              Swimming          18    50      F           9
51              Swimming          18    51      M           9
52                 Tenis          18    52      F          13
53                 Tenis          18    53      M           5
54               Theater          26    54      F          13
55               Theater          26    55      M          13
56            Volleyball          36    56      F          24
57            Volleyball          36    57      M          12

## Q6. For each department, how many students passed with a grades over 80?

## Q7. For each department, how many students passed with a grades under 60?

## Q8. Rate the teachers by their average student's grades (in descending order).

## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).